# Otimização do PSO

Aplicar a otimização utilizando o método "enxame de partículas", nesse caso, multiobjetivo.

# Bibliotecas

In [2]:
!pip install torch torchvision pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.8/303.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.6 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import time
import os
from PIL import Image

# Imports específicos para o Gerador X (Holograma)
from scipy.fft import fft2, ifft2

# Mapas de Fase
## Polarização X:

In [6]:
def load_and_preprocess_image(image_path, target_size=(450, 450)):
    """
    Carrega e pré-processa a imagem alvo usando PIL
    """
    try:
        image = Image.open(image_path).convert('L')
        image = image.resize(target_size, Image.LANCZOS)
        image_array = np.array(image, dtype=np.float64)
        image_array = image_array / np.max(image_array)
        return image_array
    except FileNotFoundError:
        print(f"Atenção: Imagem '{image_path}' não encontrada. Criando imagem de teste...")
        target_image = np.zeros(target_size)
        target_image[150:300, 100:200] = 1.0
        target_image[150:200, 200:350] = 1.0
        target_image[250:300, 200:350] = 1.0
        return target_image

def apply_zero_padding(image, padding_factor=2):
    """
    Aplica zero-padding à imagem
    """
    original_size = image.shape
    padded_size = (image.shape[0] * padding_factor, image.shape[1] * padding_factor)
    padded_image = np.zeros(padded_size, dtype=complex)

    start_row = (padded_size[0] - original_size[0]) // 2
    start_col = (padded_size[1] - original_size[1]) // 2
    padded_image[start_row:start_row+original_size[0],
                 start_col:start_col+original_size[1]] = image

    return padded_image, original_size

def create_low_pass_filter(shape, wavelength, dx, NA):
    """
    Cria filtro passa-baixa baseado na abertura numérica
    """
    nx, ny = shape
    fx = np.fft.fftfreq(nx, dx)
    fy = np.fft.fftfreq(ny, dx)
    FX, FY = np.meshgrid(fx, fy, indexing='ij')

    f_cutoff = NA / wavelength
    freq_radius = np.sqrt(FX**2 + FY**2)
    filter_mask = (freq_radius <= f_cutoff).astype(np.float64)

    return filter_mask

def angular_spectrum_propagation(U, wavelength, z, dx, filter_mask=None):
    """
    Propaga o campo usando método do espectro angular
    """
    k = 2 * np.pi / wavelength
    nx, ny = U.shape

    fx = np.fft.fftfreq(nx, dx)
    fy = np.fft.fftfreq(ny, dx)
    FX, FY = np.meshgrid(fx, fy, indexing='ij')

    root_term = 1 - (wavelength * FX)**2 - (wavelength * FY)**2
    root_term[root_term < 0] = 0

    H = np.exp(1j * k * z * np.sqrt(root_term))

    if filter_mask is not None:
        H = H * filter_mask

    # Usa as funções fft2 e ifft2 importadas da scipy.fft
    U_freq = fft2(U)
    U_prop_freq = U_freq * H
    U_prop = ifft2(U_prop_freq)

    return U_prop

def calculate_correlation(target, reconstructed):
    """
    Calcula a correlação de Pearson entre duas imagens (valores reais)
    """
    target_real = np.real(target).flatten()
    reconstructed_real = np.real(reconstructed).flatten()

    correlation = np.corrcoef(target_real, reconstructed_real)[0, 1]

    if np.isnan(correlation):
        return 0.0

    return float(correlation)

def extract_center(image, original_size):
    """
    Extrai região central da imagem (remove padding)
    """
    nx, ny = original_size
    start_row = (image.shape[0] - nx) // 2
    start_col = (image.shape[1] - ny) // 2
    return image[start_row:start_row+nx, start_col:start_col+ny]

def gerchberg_saxton_angular_spectrum(target, wavelength, z, dx, NA, num_iter=50):
    """
    Algoritmo de Gerchberg-Saxton com espectro angular
    """
    target_padded, original_size = apply_zero_padding(target)
    nx_pad, ny_pad = target_padded.shape

    filter_mask = create_low_pass_filter((nx_pad, ny_pad), wavelength, dx, NA)

    phase = np.random.rand(nx_pad, ny_pad) * 2 * np.pi
    U = target_padded * np.exp(1j * phase)

    correlations = []

    for i in range(num_iter):
        U_image = angular_spectrum_propagation(U, wavelength, z, dx, filter_mask)

        amplitude_image = np.abs(U_image)
        phase_image = np.angle(U_image)

        target_region = extract_center(target_padded, original_size)
        recon_region = extract_center(amplitude_image, original_size)

        corr = calculate_correlation(target_region, recon_region)
        correlations.append(corr)

        U_image_updated = target_padded * np.exp(1j * phase_image)

        U = angular_spectrum_propagation(U_image_updated, wavelength, -z, dx, filter_mask)

        phase_hologram = np.angle(U)
        U = np.exp(1j * phase_hologram)

        if (i + 1) % 10 == 0:
            print(f"  Iteração GS (X) {i+1}/{num_iter}, Correlação: {corr:.4f}")

    phase_final = extract_center(np.angle(U), original_size)

    return phase_final, correlations


## Polarização Y:

In [7]:
def generate_dammann_phase_map(
    P: float = 520e-9,
    wavelength: float = 1064e-9,
    supercell_pixels: int = 45,
    n_supercells: int = 10,
    iters_gs: int = 400,
    random_seed: int = 0,
    verbose: bool = True
) -> tuple[np.ndarray, dict, list]:
    """
    Gera o mapa de fase para uma grade de Dammann (spot-cloud) usando o algoritmo GS.
    """
    np.random.seed(random_seed)

    N_super = supercell_pixels
    dx = P
    d = dx * N_super

    kx = np.fft.fftfreq(N_super, d=dx)
    ky = np.fft.fftfreq(N_super, d=dx)
    kx_shift = np.fft.fftshift(kx)
    ky_shift = np.fft.fftshift(ky)
    KX, KY = np.meshgrid(kx_shift, ky_shift)
    K_rad = np.sqrt(KX**2 + KY**2)
    target_radius = min(1.0 / wavelength, 1.0 / (2.0 * dx))
    target_amp = (K_rad <= target_radius).astype(float)

    # Algoritmo GS
    plane_field = np.exp(1j * 2.0 * np.pi * np.random.rand(N_super, N_super))
    errors = []

    # Loop de iteração para Dammann
    gs_iterator = range(iters_gs)
    if verbose:
        # Cria uma barra de progresso
        gs_iterator = tqdm(range(iters_gs), desc="  Iterações GS (Y)", leave=False)

    for it in gs_iterator:
        far = np.fft.fft2(plane_field)
        far_shift = np.fft.fftshift(far)

        amp_current = np.abs(far_shift)
        err = np.sqrt(np.mean((amp_current / (amp_current.max() + 1e-9) - target_amp)**2))
        errors.append(err)

        far_shift = target_amp * np.exp(1j * np.angle(far_shift))
        far = np.fft.ifftshift(far_shift)

        plane_field = np.fft.ifft2(far)
        plane_field = np.exp(1j * np.angle(plane_field))

    supercell_phase = np.angle(plane_field)

    full_phase = np.tile(supercell_phase, (n_supercells, n_supercells))

    if verbose:
        print(f"  Mapa Dammann (Y) gerado: {full_phase.shape} pixels")

    metrics = {}

    return full_phase, metrics, errors

## Arquitetura do Simulador:
Permite que ele seja carregado e utilizado.

In [8]:
IMG_SIZE = 64
MAX_DIM_NM = 520

def draw_meta_atom_ellipse(L_x_nm, L_y_nm):
    """ Desenha um meta-átomo de ELIPSE como um array numpy binário. """
    scale_factor = IMG_SIZE / MAX_DIM_NM
    a_px = L_x_nm * scale_factor
    b_px = L_y_nm * scale_factor

    if a_px <= 0: a_px = 1e-9
    if b_px <= 0: b_px = 1e-9

    x = np.arange(0, IMG_SIZE)
    y = np.arange(0, IMG_SIZE)
    xx, yy = np.meshgrid(x, y)
    center = (IMG_SIZE - 1) / 2.0

    mask = (((xx - center) / (a_px/2.0))**2 + ((yy - center) / (b_px/2.0))**2) <= 1
    img = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)
    img[mask] = 1.0
    return img

class ResBlock(nn.Module):
    """
    Define um bloco ResNet básico
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.main_path = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        self.shortcut_path = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut_path = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    def forward(self, x):
        out = self.main_path(x) + self.shortcut_path(x)
        out = F.relu(out)
        return out

class ResNetSimulator(nn.Module):
    """
    Implementação do Simulator baseado em ResNet.
    """
    N_OUTPUTS = 4 # T_x, T_y, F_x, F_y
    def __init__(self, in_channels=1, n_outputs=N_OUTPUTS):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False), nn.BatchNorm2d(64), nn.ReLU(inplace=True))
        self.layer1 = ResBlock(64, 64, stride=1)
        self.layer2 = ResBlock(64, 128, stride=2)
        self.layer3 = ResBlock(128, 256, stride=2)
        self.layer4 = ResBlock(256, 256, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((2, 2))
        self.head = nn.Sequential(nn.Linear(256 * 2 * 2, 128), nn.ReLU(inplace=True), nn.Linear(128, n_outputs))
    def forward(self, x):
        out = self.conv1(x); out = self.layer1(out); out = self.layer2(out); out = self.layer3(out); out = self.layer4(out); out = self.avgpool(out); out = out.view(out.size(0), -1); out = self.head(out); return out


## Definição do Problema:

In [9]:
class ComplexAmplitudeProblem(Problem):
    """
    Problema de otimização que busca [L_x, L_y] para minimizar
    o erro de amplitude complexa (como no Artigo_Hugo.pdf).
    """
    def __init__(self, simulator, device,
                 target_F_x,  # Apenas a Fase X alvo
                 target_F_y): # Apenas a Fase Y alvo

        self.simulator = simulator
        self.device = device

        self.target_F_x_tensor = torch.tensor(target_F_x, device=device, dtype=torch.float32)
        self.target_F_y_tensor = torch.tensor(target_F_y, device=device, dtype=torch.float32)

        self.target_real_x = torch.cos(self.target_F_x_tensor)
        self.target_imag_x = torch.sin(self.target_F_x_tensor)
        self.target_real_y = torch.cos(self.target_F_y_tensor)
        self.target_imag_y = torch.sin(self.target_F_y_tensor)

        super().__init__(n_var=2, n_obj=2, n_constr=0,
                         xl=np.array([70, 70]), #
                         xu=np.array([200, 200])) #

    def _evaluate(self, X, out, *args, **kwargs):
        pop_size = X.shape[0]
        f1_batch = np.zeros(pop_size) # Erro X-pol
        f2_batch = np.zeros(pop_size) # Erro Y-pol

        with torch.no_grad():
            for i in range(pop_size):
                L_x_nm, L_y_nm = X[i]

                img_np = draw_meta_atom_ellipse(L_x_nm, L_y_nm)
                img_tensor = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).to(self.device)

                preds = self.simulator(img_tensor)
                pred_T_x = preds[0, 0]
                pred_T_y = preds[0, 1]
                pred_F_x = preds[0, 2]
                pred_F_y = preds[0, 3]

                pred_real_x = pred_T_x * torch.cos(pred_F_x)
                pred_imag_x = pred_T_x * torch.sin(pred_F_x)
                pred_real_y = pred_T_y * torch.cos(pred_F_y)
                pred_imag_y = pred_T_y * torch.sin(pred_F_y)

                f1 = (pred_real_x - self.target_real_x)**2 + \
                     (pred_imag_x - self.target_imag_x)**2

                f2 = (pred_real_y - self.target_real_y)**2 + \
                     (pred_imag_y - self.target_imag_y)**2

                f1_batch[i] = f1.item()
                f2_batch[i] = f2.item()

        out["F"] = np.column_stack([f1_batch, f2_batch])

## PSO Multicritério

1. Inicializa partículas aleatórias.

2. Avalia todos os objetivos.

3. Mantém um repositório de soluções não dominadas (Pareto front).

4. Move partículas em direção a líderes escolhidos do repositório.

5. Atualiza a fronteira de Pareto.

6. Repete até o número máximo de iterações.

In [12]:
def dominates(a, b):
    """Retorna True se a domina b (para minimização)."""
    return np.all(a <= b) and np.any(a < b)

def pareto_front(F):
    """Retorna os índices das soluções não-dominadas."""
    n = F.shape[0]
    mask = np.ones(n, dtype=bool)
    for i in range(n):
        for j in range(n):
            if i != j and dominates(F[j], F[i]):
                mask[i] = False
                break
    return np.where(mask)[0]

def MOPSO_custom(
    eval_func, n_particles=50, n_iter=100, dim=2, bounds=(-5, 5),
    w=0.5, c1=1.5, c2=1.5
):
    lb, ub = bounds
    # Garante que os limites sejam aplicados por dimensão se forem arrays
    if isinstance(lb, (list, np.ndarray)):
        lb = np.array(lb).reshape(1, dim)
    if isinstance(ub, (list, np.ndarray)):
        ub = np.array(ub).reshape(1, dim)

    X = np.random.uniform(lb, ub, (n_particles, dim))
    V = np.zeros_like(X)

    # Avaliação inicial
    F = eval_func(X)
    pbest = X.copy()
    F_pbest = F.copy()

    # Arquivo (repositório Pareto)
    idx_pareto = pareto_front(F)
    archive_X = X[idx_pareto]
    archive_F = F[idx_pareto]

    for it in range(n_iter):
        # Escolher líder aleatório do repositório
        leaders_idx = np.random.choice(len(archive_X), n_particles)
        G = archive_X[leaders_idx]

        # Atualizar velocidade e posição
        r1, r2 = np.random.rand(n_particles, dim), np.random.rand(n_particles, dim)
        V = w*V + c1*r1*(pbest - X) + c2*r2*(G - X)
        X = np.clip(X + V, lb, ub)

        F = eval_func(X)

        for i in range(n_particles):
            if dominates(F[i], F_pbest[i]):
                pbest[i], F_pbest[i] = X[i].copy(), F[i].copy()

        all_X = np.vstack([archive_X, X])
        all_F = np.vstack([archive_F, F])
        idx = pareto_front(all_F)
        archive_X, archive_F = all_X[idx], all_F[idx]

        if len(archive_X) > 100:
            keep = np.random.choice(len(archive_X), 100, replace=False)
            archive_X, archive_F = archive_X[keep], archive_F[keep]

    return archive_X, archive_F

def evaluate_pixel_objectives(X_batch, simulator, device, t_F_x, t_F_y):
    """
    Avalia um batch de partículas [L_x, L_y] para um *único* pixel alvo (t_F_x, t_F_y).
    """
    pop_size = X_batch.shape[0]
    f_batch = np.zeros((pop_size, 2)) # (Erro_X, Erro_Y)

    # Pré-calcular alvos no device (para eficiência)
    target_real_x = torch.cos(torch.tensor(t_F_x, device=device))
    target_imag_x = torch.sin(torch.tensor(t_F_x, device=device))
    target_real_y = torch.cos(torch.tensor(t_F_y, device=device))
    target_imag_y = torch.sin(torch.tensor(t_F_y, device=device))

    with torch.no_grad():
        for i in range(pop_size):
            L_x_nm, L_y_nm = X_batch[i]

            img_np = draw_meta_atom_ellipse(L_x_nm, L_y_nm)
            img_tensor = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).to(device)

            # Passa pelo simulador
            preds = simulator(img_tensor)
            pred_T_x, pred_T_y, pred_F_x, pred_F_y = preds[0]

            # Calcula os campos complexos (previsto vs. alvo)
            pred_real_x = pred_T_x * torch.cos(pred_F_x)
            pred_imag_x = pred_T_x * torch.sin(pred_F_x)
            pred_real_y = pred_T_y * torch.cos(pred_F_y)
            pred_imag_y = pred_T_y * torch.sin(pred_F_y)

            # Calcula os erros (objetivos f1 e f2)
            f1 = (pred_real_x - target_real_x)**2 + (pred_imag_x - target_imag_x)**2
            f2 = (pred_real_y - target_real_y)**2 + (pred_imag_y - target_imag_y)**2

            f_batch[i, 0] = f1.item()
            f_batch[i, 1] = f2.item()

    return f_batch

In [ ]:
# ===================================================================
# --- Bloco 4: Execução Principal (Otimização) ---
# ===================================================================
if __name__ == "__main__":

    # --- 1. Parâmetros Fixos ---
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Parâmetros de Geração (X - Holograma)
    IMG_ALVO_X = '/content/WhatsApp Image 2025-11-06 at 16.18.47.png' # Nome do arquivo de imagem
    TAMANHO_ALVO_X = (45, 45) # Tamanho final da metasuperfície
    PARAMS_GS_X = {
        'wavelength': 1064e-9, #
        'z': 380e-6,           #
        'dx': 520e-9,          #
        'NA': 0.65,            #
        'num_iter': 50         #
    }

    # Parâmetros de Geração (Y - Dammann)
    PARAMS_GS_Y = {
        'P': 520e-9,                       #
        'wavelength': 1064e-9,
        'supercell_pixels': 45,
        'n_supercells': TAMANHO_ALVO_X[0] // 45, # (450 // 45 = 10)
        'iters_gs': 400,
        'verbose': True
    }

    CAMINHO_SIMULADOR = '/content/simulator_teste_2,.pth' #
    POP_SIZE_AG = 20
    GERACOES_AG = 40

    # Geração dos Mapas de Fase
    print(f"Usando dispositivo: {device}")

    print(f"\n--- Etapa 1: Gerando Fase X (Holograma) de '{IMG_ALVO_X}' ---")
    target_image_x = load_and_preprocess_image(IMG_ALVO_X, target_size=TAMANHO_ALVO_X)
    target_map_Fase_X, _ = gerchberg_saxton_angular_spectrum(
        target_image_x, **PARAMS_GS_X
    )
    MAP_H, MAP_W = target_map_Fase_X.shape
    print(f"Mapa X gerado: {target_map_Fase_X.shape}")

    print(f"\n--- Etapa 2: Gerando Fase Y (Dammann) ---")
    target_map_Fase_Y, _, _ = generate_dammann_phase_map(**PARAMS_GS_Y)

    if target_map_Fase_Y.shape != (MAP_H, MAP_W):
        print(f"ERRO: O shape do mapa Y {target_map_Fase_Y.shape} não bate com o do mapa X {(MAP_H, MAP_W)}!")
        exit()

    # Carregar Simulador
    print(f"\n--- Etapa 3: Carregando simulador de '{CAMINHO_SIMULADOR}' ---")
    if not os.path.exists(CAMINHO_SIMULADOR):
        print(f"ERRO: Arquivo do simulador '{CAMINHO_SIMULADOR}' não encontrado.")
        print("Por favor, treine o simulador (parte 1 do notebook) primeiro.")
        exit()

    simulator = ResNetSimulator(in_channels=1, n_outputs=4).to(device)
    simulator.load_state_dict(torch.load(CAMINHO_SIMULADOR, map_location=device))
    simulator.eval()
    print("Simulador carregado.")

    print(f"\n--- Etapa 4: Iniciando otimização pixel-a-pixel (com MOPSO Customizado) ---")
    print(f"Tamanho do mapa: {MAP_H}x{MAP_W} ({MAP_H*MAP_W} pixels)")

    POP_SIZE_PSO = POP_SIZE_AG
    ITERACOES_PSO = GERACOES_AG

    print(f"Parâmetros PSO: {POP_SIZE_PSO} partículas, {ITERACOES_PSO} iterações por pixel")

    metasurface_design_Lx = np.zeros((MAP_H, MAP_W))
    metasurface_design_Ly = np.zeros((MAP_H, MAP_W))
    metasurface_pixel_errors = np.zeros((MAP_H, MAP_W, 2)) # 2 objetivos de erro

    # (Não precisamos mais do 'algorithm', 'termination' ou 'ComplexAmplitudeProblem' do pymoo)

    start_time_total = time.time()

    for r in tqdm(range(MAP_H), desc="Otimizando Linhas"):
        for c in range(MAP_W):
            t_F_x = target_map_Fase_X[r, c]
            t_F_y = target_map_Fase_Y[r, c]

            # 1. Definir os limites (bounds)
            #    (Baseado nos seus limites [70, 70] e [200, 200] do Pymoo)
            lim_inferior = np.array([70, 70])
            lim_superior = np.array([200, 200])

            # 2. Criar a função de avaliação "on-the-fly"
            #    Usamos uma função lambda para "capturar" os alvos (t_F_x, t_F_y)
            #    e passá-los para a nossa função "ponte".
            #    (cap_... = captura o valor atual do loop)
            eval_func_pixel = lambda x_batch, cap_t_F_x=t_F_x, cap_t_F_y=t_F_y: evaluate_pixel_objectives(
                x_batch, simulator, device, cap_t_F_x, cap_t_F_y
            )

            # 3. Chamar o seu MOPSO
            pareto_X, pareto_F = MOPSO_custom(
                eval_func=eval_func_pixel,
                n_particles=POP_SIZE_PSO,
                n_iter=ITERACOES_PSO,
                dim=2, # (L_x, L_y)
                bounds=(lim_inferior, lim_superior), # Passando os limites
                w=0.5, c1=1.5, c2=1.5 # (Parâmetros padrão do seu PSO)
            )

            # 4. Selecionar o melhor da fronteira (exatamente a mesma lógica)
            errors = pareto_F
            norms = np.linalg.norm(errors, axis=1) # Distância Euclidiana 2D
            best_index = np.argmin(norms)
            best_geometry = pareto_X[best_index] # [L_x, L_y]
            best_errors = pareto_F[best_index]   # [Error_X, Error_Y]

            # 5. Armazenar resultados
            metasurface_design_Lx[r, c] = best_geometry[0]
            metasurface_design_Ly[r, c] = best_geometry[1]
            metasurface_pixel_errors[r, c, :] = best_errors

    end_time_total = time.time()
    total_time_min = (end_time_total - start_time_total) / 60
    print(f"\nOtimização de {MAP_H*MAP_W} pixels concluída em {total_time_min:.2f} minutos.")

    # --- 5. Salvar Resultados ---
    print("Salvando design final...")
    np.save("metasurface_design_Lx_final.npy", metasurface_design_Lx)
    np.save("metasurface_design_Ly_final.npy", metasurface_design_Ly)
    np.save("metasurface_pixel_errors_final.npy", metasurface_pixel_errors)

    print("Resultados salvos em 'metasurface_design_Lx_final.npy' e 'metasurface_design_Ly_final.npy'")

    # --- 6. Gerar e Salvar Gráficos ---
    print("\n--- Etapa 5: Gerando gráficos dos resultados ---")

    # Carrega os resultados salvos
    final_Lx = np.load("metasurface_design_Lx_final.npy")
    final_Ly = np.load("metasurface_design_Ly_final.npy")
    final_errors = np.load("metasurface_pixel_errors_final.npy")


    # --- Gráfico 1: Design Físico (Lx e Ly) ---
    # (Similar a image_1d08c0.jpg)

    # Usa os limites do otimizador (xl, xu) para definir a escala de cores
    vmin_val = 70
    vmax_val = 200

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle("Design Físico Final da Metassuperfície", fontsize=16)

    # Plot L_x
    im1 = ax1.imshow(final_Lx, cmap='viridis', vmin=vmin_val, vmax=vmax_val)
    ax1.set_title("Distribuição de L_x")
    ax1.set_xlabel("Pixel (coluna)")
    ax1.set_ylabel("Pixel (linha)")
    fig.colorbar(im1, ax=ax1, label="Comprimento Lx (nm)")

    # Plot L_y
    im2 = ax2.imshow(final_Ly, cmap='viridis', vmin=vmin_val, vmax=vmax_val)
    ax2.set_title("Distribuição de L_y")
    ax2.set_xlabel("Pixel (coluna)")
    fig.colorbar(im2, ax=ax2, label="Comprimento Ly (nm)")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig("plot_design_fisico_final.png")
    print("Gráfico 'plot_design_fisico_final.png' salvo.")


    # --- Gráfico 2: Mapa de Erro ---
    # (Similar a image_1d07e2.jpg)

    # O arquivo de erro tem shape (H, W, 2). Calculamos a norma Euclidiana
    # (distância) para ter um único valor de erro por pixel.
    error_norm = np.linalg.norm(final_errors, axis=2)

    plt.figure(figsize=(7, 6))
    plt.imshow(error_norm, cmap='afmhot') # 'afmhot' é similar ao 'hot' da sua imagem
    plt.title("Mapa de Erro do Casamento de Fase")
    plt.xlabel("Pixel (coluna)")
    plt.ylabel("Pixel (linha)")
    plt.colorbar(label="Erro (Norma Euclidiana)")
    plt.tight_layout()
    plt.savefig("plot_mapa_erro_final.png")
    print("Gráfico 'plot_mapa_erro_final.png' salvo.")


    # --- Gráfico 3: Validação da Reconstrução ---
    # (Similar a image_1d0539.jpg)

    # Para criar este gráfico, precisamos simular a propagação
    # usando o design final (Lx, Ly) que acabamos de otimizar.

    print("Iniciando validação da reconstrução (pode levar um momento)...")

    # Prepara arrays para guardar a fase e amplitude reais do design
    achieved_phase_X = np.zeros((MAP_H, MAP_W))
    achieved_amplitude_X = np.zeros((MAP_H, MAP_W))

    # Garante que o simulador está em modo de avaliação
    simulator.eval()
    with torch.no_grad():
        # Itera por cada pixel do design final
        for r in tqdm(range(MAP_H), desc="  Validando design", leave=False):
            for c in range(MAP_W):
                L_x_nm = final_Lx[r, c]
                L_y_nm = final_Ly[r, c]

                # Desenha o meta-átomo e passa pelo simulador (ResNet)
                img_np = draw_meta_atom_ellipse(L_x_nm, L_y_nm)
                img_tensor = torch.from_numpy(img_np).unsqueeze(0).unsqueeze(0).to(device)
                preds = simulator(img_tensor)

                # Armazena a amplitude (T_x) e fase (F_x) previstas
                achieved_amplitude_X[r, c] = preds[0, 0].item()
                achieved_phase_X[r, c] = preds[0, 2].item()

    # Cria o campo complexo no plano da metassuperfície
    U_start_plane = achieved_amplitude_X * np.exp(1j * achieved_phase_X)

    # Aplica padding
    U_padded, original_size = apply_zero_padding(U_start_plane)
    nx_pad, ny_pad = U_padded.shape

    # Pega os parâmetros de propagação da Etapa 1
    params_x = PARAMS_GS_X

    # Cria o filtro passa-baixa (exatamente como no algoritmo GS)
    filter_mask = create_low_pass_filter(
        (nx_pad, ny_pad),
        params_x['wavelength'],
        params_x['dx'],
        params_x['NA']
    )

    # Propaga o campo usando o Espectro Angular
    U_image_reconstructed = angular_spectrum_propagation(
        U_padded,
        params_x['wavelength'],
        params_x['z'],
        params_x['dx'],
        filter_mask
    )

    # Extrai a amplitude da região central (remove o padding)
    recon_amplitude_final = np.abs(extract_center(U_image_reconstructed, original_size))

    # Plota a imagem alvo original (que já está na memória) vs. a reconstrução
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle("Validação Final da Reconstrução", fontsize=16)

    # Imagem Alvo (target_image_x foi carregada na Etapa 1)
    ax1.imshow(target_image_x, cmap='gray')
    ax1.set_title(f"Imagem Alvo Original ({MAP_H}x{MAP_W})")

    # Imagem Reconstruída
    ax2.imshow(recon_amplitude_final, cmap='gray')
    ax2.set_title(f"Imagem Reconstruída ({MAP_H}x{MAP_W})")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig("plot_validacao_reconstrucao.png")
    print("Gráfico 'plot_validacao_reconstrucao.png' salvo.")
    print("--- Geração de gráficos concluída! ---")

    print("Script concluído.")

Usando dispositivo: cpu

--- Etapa 1: Gerando Fase X (Holograma) de '/content/WhatsApp Image 2025-11-06 at 16.18.47.png' ---
  Iteração GS (X) 10/50, Correlação: 0.6094
  Iteração GS (X) 20/50, Correlação: 0.6418
  Iteração GS (X) 30/50, Correlação: 0.6569
  Iteração GS (X) 40/50, Correlação: 0.6725
  Iteração GS (X) 50/50, Correlação: 0.6760
Mapa X gerado: (45, 45)

--- Etapa 2: Gerando Fase Y (Dammann) ---


  Mapa Dammann (Y) gerado: (45, 45) pixels

--- Etapa 3: Carregando simulador de '/content/simulator_teste_2,.pth' ---
Simulador carregado.

--- Etapa 4: Iniciando otimização pixel-a-pixel (com MOPSO Customizado) ---
Tamanho do mapa: 45x45 (2025 pixels)
Parâmetros PSO: 20 partículas, 40 iterações por pixel


Otimizando Linhas:   0%|          | 0/45 [00:00<?, ?it/s]